In [1]:
import torch
from torchinfo import summary
import s3prl.hub as hub

################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################



Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



In [7]:
model = getattr(hub, "mockingjay")()  # build the Mockingjay model with pre-trained weights
print(model)

Using cache found in /home/chihyuan/.cache/torch/hub/s3prl_cache/a2b432be9adba2cb59f5cf89a4cf84d5fff8ec3c9fe248ad53349694565ef8c9
for https://www.dropbox.com/s/zwsfa6w2iy2cc68/states-500000.ckpt?dl=0
[UpstreamExpert] - Using the default upstream expert config
UpstreamExpert(
  (transformer): PretrainedTransformer(
    (extracter): OnlinePreprocessor(
      (_melscale): MelScale()
      (_mfcc_trans): MFCC(
        (amplitude_to_DB): AmplitudeToDB()
        (MelSpectrogram): MelSpectrogram(
          (spectrogram): Spectrogram()
          (mel_scale): MelScale()
        )
      )
    )
    (model): TransformerModel(
      (shared_parameters): ModuleDict()
      (invertible_adapters): ModuleDict()
      (input_representations): TransformerInputRepresentations(
        (spec_transform): Linear(in_features=80, out_features=768, bias=True)
        (LayerNorm): TransformerLayerNorm()
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): TransformerEncoder(
        (layer)

In [3]:
summary(model)

Layer (type:depth-idx)                                                 Param #
UpstreamExpert                                                         --
├─PretrainedTransformer: 1-1                                           --
│    └─OnlinePreprocessor: 2-1                                         --
│    │    └─MelScale: 3-1                                              --
│    │    └─MFCC: 3-2                                                  --
│    └─TransformerModel: 2-2                                           --
│    │    └─ModuleDict: 3-3                                            --
│    │    └─ModuleDict: 3-4                                            --
│    │    └─TransformerInputRepresentations: 3-5                       63,744
│    │    └─TransformerEncoder: 3-6                                    85,054,464
│    │    └─PrefixTuningPool: 3-7                                      --
Total params: 85,118,208
Trainable params: 85,118,208
Non-trainable params: 0

In [4]:
type(model.transformer.model)  # the model with adpaters' functions

s3prl.upstream.mockingjay.model.TransformerModel

In [5]:
model.transformer.model.config

TransformerConfig {
  "adapters": {
    "adapters": {},
    "config_map": {},
    "fusion_config_map": {},
    "fusions": {}
  },
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "model_type": "transformer",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pre_layer_norm": false,
  "share_layer": false,
  "transformers_version": "4.17.0"
}

In [8]:
model.transformer.model.add_adapter("acu", "houlsby")
print(model)

UpstreamExpert(
  (transformer): PretrainedTransformer(
    (extracter): OnlinePreprocessor(
      (_melscale): MelScale()
      (_mfcc_trans): MFCC(
        (amplitude_to_DB): AmplitudeToDB()
        (MelSpectrogram): MelSpectrogram(
          (spectrogram): Spectrogram()
          (mel_scale): MelScale()
        )
      )
    )
    (model): TransformerModel(
      (shared_parameters): ModuleDict()
      (invertible_adapters): ModuleDict()
      (input_representations): TransformerInputRepresentations(
        (spec_transform): Linear(in_features=80, out_features=768, bias=True)
        (LayerNorm): TransformerLayerNorm()
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): TransformerEncoder(
        (layer): ModuleList(
          (0): TransformerLayer(
            (attention): TransformerAttention(
              (self): TransformerSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_feature

In [9]:
model.transformer.model.train_adapter("acu")
summary(model)

Layer (type:depth-idx)                                                                     Param #
UpstreamExpert                                                                             --
├─PretrainedTransformer: 1-1                                                               --
│    └─OnlinePreprocessor: 2-1                                                             --
│    │    └─MelScale: 3-1                                                                  --
│    │    └─MFCC: 3-2                                                                      --
│    └─TransformerModel: 2-2                                                               --
│    │    └─ModuleDict: 3-3                                                                --
│    │    └─ModuleDict: 3-4                                                                --
│    │    └─TransformerInputRepresentations: 3-5                                           (63,744)
│    │    └─TransformerEncoder: 3-6              

In [10]:
summary(model, depth=20, row_settings=["depth", "var_names"])

Layer (type (var_name):depth-idx)                                                                    Param #
UpstreamExpert                                                                                       --
├─PretrainedTransformer (transformer): 1-1                                                           --
│    └─OnlinePreprocessor (extracter): 2-1                                                           --
│    │    └─MelScale (_melscale): 3-1                                                                --
│    │    └─MFCC (_mfcc_trans): 3-2                                                                  --
│    │    │    └─AmplitudeToDB (amplitude_to_DB): 4-1                                                --
│    │    │    └─MelSpectrogram (MelSpectrogram): 4-2                                                --
│    │    │    │    └─Spectrogram (spectrogram): 5-1                                                 --
│    │    │    │    └─MelScale (mel_scale): 5-2            

### Psuedo Input

In [9]:
model.transformer.model.active_adapters

Stack[acu]

In [10]:
# model.transformer.model.add_adapter("acu", "pfeiffer")  # Before activated, resul is the same as original mockingjay in original s3prl
wavs = [torch.ones(160000, dtype=torch.float) for _ in range(16)]
model.train()  # will have random effect
# model.eval()
with torch.no_grad():
    print(model(wavs))

{'last_hidden_state': tensor([[[ 6.7645e-04, -3.3540e-01,  5.6363e-01,  ...,  4.5578e-01,
           3.2929e-01, -5.7725e-01],
         [ 3.7545e-02, -4.7331e-01,  5.7534e-01,  ...,  5.3402e-02,
           3.6354e-01, -3.7674e-01],
         [ 5.2452e-02, -3.6531e-01,  5.9790e-01,  ...,  3.6646e-01,
           3.2378e-01, -4.2081e-01],
         ...,
         [ 7.9032e-02, -3.8343e-01,  6.1830e-01,  ...,  3.4670e-01,
           3.7949e-01, -5.2737e-01],
         [ 2.0494e-01, -3.9395e-01,  5.7528e-01,  ...,  1.6519e-01,
           3.3090e-01, -5.2240e-01],
         [ 1.6175e-01, -3.0023e-01,  7.3612e-01,  ...,  2.8633e-01,
           3.2639e-01, -4.8198e-01]],

        [[ 1.1004e-01, -2.5976e-01,  5.8541e-01,  ...,  2.2522e-01,
           2.8089e-01, -4.5776e-01],
         [ 9.8611e-02, -3.7397e-01,  1.3657e-01,  ...,  8.2264e-02,
           4.2774e-01, -3.3455e-01],
         [ 7.6987e-02, -3.9069e-01,  2.4916e-01,  ...,  1.0845e-01,
           4.1700e-01, -3.0493e-01],
         ...,
   

### utils for testing

In [6]:
from s3prl.adapters.layer import AdapterLayer, AdapterLayerBase
from s3prl.adapters.configuration import AdapterConfig
for (i, layer) in model.transformer.model.iter_layers():
    for module in layer.modules():
        if (isinstance(module, AdapterLayerBase)):
            print(module.config.adapters.match(
                "acu",
                config_type=AdapterConfig,
                layer_idx=i,
                location_key="self",
            ))
            print(module.config.adapters.to_dict())
    break

None
{'adapters': {}, 'config_map': {}, 'fusions': {}, 'fusion_config_map': {}}
None
{'adapters': {}, 'config_map': {}, 'fusions': {}, 'fusion_config_map': {}}
None
{'adapters': {}, 'config_map': {}, 'fusions': {}, 'fusion_config_map': {}}


In [3]:
model.transformer.model.config.hahaha = "hahaha"
model.transformer.model.config

TransformerConfig {
  "adapters": {
    "adapters": {},
    "config_map": {},
    "fusion_config_map": {},
    "fusions": {}
  },
  "attention_probs_dropout_prob": 0.1,
  "hahaha": "hahaha",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "model_type": "transformer",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pre_layer_norm": false,
  "share_layer": false,
  "transformers_version": "4.17.0"
}

In [6]:
model.transformer.model.encoder.layer[0].output.config

TransformerConfig {
  "adapters": {
    "adapters": {},
    "config_map": {},
    "fusion_config_map": {},
    "fusions": {}
  },
  "attention_probs_dropout_prob": 0.1,
  "hahaha": "hahaha",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "model_type": "transformer",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pre_layer_norm": false,
  "share_layer": false,
  "transformers_version": "4.17.0"
}

In [5]:
model.transformer.model.base_model.encoder.layer[0].output.config

TransformerConfig {
  "adapters": {
    "adapters": {},
    "config_map": {},
    "fusion_config_map": {},
    "fusions": {}
  },
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "model_type": "transformer",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pre_layer_norm": false,
  "share_layer": false,
  "transformers_version": "4.17.0"
}

In [5]:
model.transformer.model.add_adapter("acu")
print(model.transformer.model.config.adapters.to_dict())
print(model)

2022-05-24 19:53:17 | INFO | s3prl.adapters.configuration | Adding adapter 'acu'.


{'adapters': {'acu': 'pfeiffer'}, 'config_map': {}, 'fusions': {}, 'fusion_config_map': {}}
UpstreamExpert(
  (transformer): PretrainedTransformer(
    (extracter): OnlinePreprocessor(
      (_melscale): MelScale()
      (_mfcc_trans): MFCC(
        (amplitude_to_DB): AmplitudeToDB()
        (MelSpectrogram): MelSpectrogram(
          (spectrogram): Spectrogram()
          (mel_scale): MelScale()
        )
      )
    )
    (model): TransformerModel(
      (shared_parameters): ModuleDict()
      (invertible_adapters): ModuleDict()
      (input_representations): TransformerInputRepresentations(
        (spec_transform): Linear(in_features=80, out_features=768, bias=True)
        (LayerNorm): TransformerLayerNorm()
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): TransformerEncoder(
        (layer): ModuleList(
          (0): TransformerLayer(
            (attention): TransformerAttention(
              (self): TransformerSelfAttention(
                (query): L